In [ ]:
# Шаг 0: Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Шаг 1: Загрузка данных
df = pd.read_csv("anime.csv")

# Шаг 2: Убираем строки без жанра или рейтинга
df = df[df['genre'].notna()]
df = df[df['rating'].notna()]

# Шаг 3: Сохраняем имена аниме (для поиска)
df['name'] = df['name'].astype(str)

# Шаг 4: Оставляем нужные признаки
df = df[['name', 'genre', 'type', 'episodes', 'members', 'rating']]

# Шаг 5: Обрабатываем жанры (genre — строка: "Action, Fantasy")
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
genre_matrix = vectorizer.fit_transform(df['genre'])

# Шаг 6: Преобразуем "type" (TV, Movie и т.п.) в числа
df['type'] = df['type'].astype('category')
df['type_code'] = df['type'].cat.codes

# Шаг 7: Обрабатываем episodes (строки → числа + замена NaN на медиану)
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
df['episodes'] = df['episodes'].fillna(df['episodes'].median())

# Шаг 8: Подготовка признаков (X)
X_other = df[['type_code', 'episodes', 'members']].values
X_all = np.hstack((genre_matrix.toarray(), X_other))

# Шаг 9: Целевая переменная (y)
y = df['rating'].values

# Шаг 10: Делим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)

# Шаг 11: Обучаем модель
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Шаг 12: Предсказание и ошибка
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
# print("Среднеквадратичная ошибка (MSE):", round(mse, 2))

# Шаг 13: Функция для предсказания по названию
def predict_anime_rating(anime_name):
    results = df[df['name'].str.lower() == anime_name.lower()]

    if results.empty:
        print(f"❌ Аниме с названием '{anime_name}' не найдено.")
        return

    for idx, row in results.iterrows():
        # Преобразуем жанры в вектор
        genre_vec = vectorizer.transform([row['genre']]).toarray()

        # Признаки: type_code, episodes, members
        other_features = np.array([[row['type_code'], row['episodes'], row['members']]])

        # Объединяем всё
        full_features = np.hstack((genre_vec, other_features))

        # Предсказание
        predicted_rating = model.predict(full_features)[0]

        # Результат
        print(f"\n🎬 Название: {row['name']}")
        print(f"📊 Реальный рейтинг: {row['rating']}")
        print(f"🤖 Предсказанный рейтинг: {round(predicted_rating, 2)}")

# Шаг 14: Пример использования
while True:
    anime_input = input("\nВведите название аниме (или '0' для завершения): ")
    if anime_input.lower() == '0':
        break
    predict_anime_rating(anime_input)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



Введите название аниме (или '0' для завершения):  Monster



🎬 Название: Monster
📊 Реальный рейтинг: 8.72
🤖 Предсказанный рейтинг: 8.62
